<a href="https://colab.research.google.com/github/flatfeetpete/nerfstudio/blob/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@markdown <h1>Install Conda (requires runtime restart)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
#@markdown <h1>Install Nerfstudio and Dependencies (~15 min)</h1>

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=12RL_NVgE9WGvr_fEsXEiuaJ1QESvQCPl&confirm=t" 
!pip install tinycudann-1.7-cp38-cp38-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
# !pip install nerfstudio
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git
!conda remove --force qt-main

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDE9C dozer' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content
--2023-02-26 20:21:20--  https://data.nerf.studio/nerfstudio/dozer.zip
Resolving data.nerf.studio (data.nerf.studio)... 34.102.68.79
Connecting to data.nerf.studio (data.nerf.studio)|34.102.68.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1347483375 (1.3G) [application/zip]
Saving to: ‘data/nerfstudio/dozer.zip’

data/nerfstudio/doz 100%[===================>]   1.25G  1.74MB/s    in 4m 13s  

2023-02-26 20:25:34 (5.08 MB/s) - ‘data/nerfstudio/dozer.zip’ saved [1347483375/1347483375]

Data Processing Succeeded!


In [37]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.037s
https://viewer.nerf.studio/?websocket_url=wss://solid-ends-buy-35-240-238-239.loca.lt
You may need to click Refresh Page after you start training!


In [9]:
#@markdown <h1>Start Training</h1>

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

Streaming output truncated to the last 5000 lines.
Viewer at: https://viewer.nerf.studio/versions/23-02-3-0/?websocket_url=ws://localhost:7007              
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec     Vis Rays / Sec        
-------------------------------------------------------------------------------------------------------- 
26380 (87.93%)      51.943 ms            3 m, 8 s             81.15 K                                    
26390 (87.97%)      50.170 ms            3 m, 1 s             83.60 K                                    
26400 (88.00%)      49.091 ms            2 m, 56 s            85.17 K              351.83 K              
26410 (88.03%)      49.056 ms            2 m, 56 s            85.35 K                                    
26420 (88.07%)      49.978 ms            2 m, 58 s            84.18 K              333.42 K              
26430 (88.10%)      49.824 ms            2 m, 57 s            84.32 K                                

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
# This restarts training, but only works if you finished training on your first run. You can use the UI to continue traiing if it didn't finish.
%cd /content
#!echo $(ls -rt outputs/data-nerfstudio-$scene/nerfacto | head -1)
#!ns-train nerfacto --viewer.websocket-port 7007 --load-dir=outputs/data-nerfstudio-$scene/nerfacto/$(ls -rt outputs/data-nerfstudio-$scene/nerfacto | head -1)/nerfstudio_models --viewer.start-train=False nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

!ns-train nerfacto --viewer.websocket-port 7007 --load-dir=/content/outputs/unnamed/nerfacto/2023-03-01_034009/nerfstudio_models --viewer.start-train=False nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4



/content
──────────────────────────────────────────────────────── Config ────────────────────────────────────────────────────────
TrainerConfig(
    _target=<class 'nerfstudio.engine.trainer.Trainer'>,
    output_dir=PosixPath('outputs'),
    method_name='nerfacto',
    experiment_name=None,
    timestamp='2023-03-01_065124',
    machine=MachineConfig(seed=42, num_gpus=1, num_machines=1, machine_rank=0, dist_url='auto'),
    logging=LoggingConfig(
        relative_log_dir=PosixPath('.'),
        steps_per_log=10,
        max_buffer_size=20,
        local_writer=LocalWriterConfig(
            _target=<class 'nerfstudio.utils.writer.LocalWriter'>,
            enable=True,
            stats_to_track=(
                <EventName.ITER_TRAIN_TIME: 'Train Iter (time)'>,
                <EventName.TRAIN_RAYS_PER_SEC: 'Train Rays / Sec'>,
                <EventName.CURR_TEST_PSNR: 'Test PSNR'>,
                <EventName.VIS_RAYS_PER_SEC: 'Vis Rays / Sec'>,
                <EventName.TEST_RAYS_

In [42]:
%cd /content/nerf-work/
!git pull
%cd /content/nerfstudio/
!git pull

/content/nerf-work
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 25.19 KiB | 4.20 MiB/s, done.
From github.com:flatfeetpete/nerf-work
   5a5b561..c834913  main       -> origin/main
Updating 5a5b561..c834913
Fast-forward
 2023-03-01_065124.json | 4258 +++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4257 insertions(+), 1 deletion(-)
/content/nerfstudio
Already up to date.


In [65]:
import datetime
import pytz
%cd /content/nerf-work/
for i in range(60):
  !git add --all
  !git commit -a -m 'dummy commit message (likely test files)'
  !git push
  print(datetime.datetime.now(pytz.timezone('America/Los_Angeles')))
  time.sleep(300)


/content/nerf-work
[main 6bf3803] dummy commit message (likely test files)
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 20230228-long-dragon-0.03.tagged.looped.mp4
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 104.26 MiB | 2.46 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: error: Trace: f603acf41df83fb26bcc659d6870058dca989bb1f415cc01d00a1a80789efc89
remote: error: See http://git.io/iEPt8g for more information.
remote: error: File 20230228-long-dragon-0.03.tagged.looped.mp4 is 104.24 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To github.com:flatfeetpete/nerf-work.git
 ! [remote rejected] main -> main (pre-receive hook declined)
er

KeyboardInterrupt: ignored

In [ ]:
# !ns-render --load-config outputs/data-nerfstudio-dozer/nerfacto/2023-02-28_010246/config.yml --traj filename --camera-path-filename data/nerfstudio/dozer/camera_paths/2023-02-28_010246.json --output-path renders/dozer/2023-02-28_010246.mp4
#%cd /content
#!cp data/nerfstudio/dozer/camera_paths/2023-02-28_010246.json nerf-work/longer_path.json
# --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-01_065124.json --output-path renders/custom_data/2023-03-01_065124.mp4

export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/render.py --load-config outputs/unnamed/nerfacto/2023-03-01_065124/config.yml --traj filename --camera-path-filename nerf-work/2023-03-01_065124.json --output-path /content/nerf-work/20230228-long-dragon-0.03.mp4

#export PYTHONPATH=/content/nerfstudio/:/env/python; python3 /content/nerfstudio/scripts/render.py --load-config outputs/unnamed/nerfacto/2023-03-01_034009/config.yml --traj filename --camera-path-filename data/nerfstudio/custom_data/camera_paths/2023-03-01_034009.json --output-path /content/nerf-works/20230228-dragon-lowfps.mp4


/content


In [66]:
%cd /content
# WORKED !ffmpeg -i /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.mp4 -vcodec libx264 -x264opts "frame-packing=3" /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.sbsembed.mp4

!ffmpeg -stream_loop 3 -i /content/nerf-work/20230228-long-dragon-0.03.mp4 -vcodec libx264 -x264opts "frame-packing=3" /content/nerf-work/20230228-long-dragon-0.03.tagged.looped3.mp4

# FAILED  !ffmpeg -i /content/nerf-work/20230227_dozer_2d_sbs_orbit_ed0.05.mp4 -c copy -metadata:s:v:0 stereo_mode=1 /content/nerf-work/20230227_sbs001.mkv 

/content
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libv

In [34]:
%cd /content/nerfstudio
!git commit -a -m "fix ipd hacks"
!git push

/content/nerfstudio
[main 75decf4] fix ipd hacks
 1 file changed, 2 insertions(+), 1 deletion(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 449 bytes | 449.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To github.com:flatfeetpete/nerfstudio.git
   31a827d..75decf4  main -> main
